<center>
<table>
  <tr>
    <td><img src="https://portal.nccs.nasa.gov/datashare/astg/training/python/logos/nasa-logo.svg" width="100"/> </td>
     <td><img src="https://portal.nccs.nasa.gov/datashare/astg/training/python/logos/ASTG_logo.png?raw=true" width="80"/> </td>
     <td> <img src="https://www.nccs.nasa.gov/sites/default/files/NCCS_Logo_0.png" width="130"/> </td>
    </tr>
</table>
</center>

        
<center>
<h1><font color= "blue" size="+3">ASTG Python Courses</font></h1>
</center>

---

<CENTER>
<H1> <font color="red" size="+3">
    Web Scraping with Python </font> 
    <br>
    Helio Applications
</H1>
</CENTER>

## <font color='red'>Reference Documents</font>

- [Web Scraping: What It Is and How to Use It](https://scrape-it.cloud/blog/web-scraping-what-it-is-and-how-to-use-it)
- [What is web scraping](https://www.scrapehero.com/a-beginners-guide-to-web-scraping-part-1-the-basics/)
- [Python Requests Tutorial](http://zetcode.com/python/requests/)
- [Python’s Requests Library (Guide](https://realpython.com/python-requests/)
- [Building a Web Scraper from start to finish](https://hackernoon.com/building-a-web-scraper-from-start-to-finish-bb6b95388184)
- [Visualizing Asteroids in Python with Bokeh and NASA APIs](https://hackernoon.com/visualizing-asteroids-in-python-with-bokeh-and-nasa-apis-4j3l3tfa) by AShley Wans, October 2020.
- [Space Weather Database Of Notifications, Knowledge, Information (DONKI)](https://kauai.ccmc.gsfc.nasa.gov/DONKI/)
- [NasaPy](https://nasapy.readthedocs.io/en/latest/)

## <font color='red'>Objectives</font>

In this presentation:

- We use web scraping procedures to extract useful data from websites containing Helio related information. The website contents have JSON and HTML formats.
- We focus on accessing the Space Weather Database Of Notifications, Knowledge, Information (DONKI) database to extract Coronal Mass Ejection (CME) activities over a specified date range.
The collected data is processed for analysis and visualization.

![fig_web](https://cdn.prod.website-files.com/65d8ee5f025f02594c614c17/66e52b7b03cd70fb09199a8a_65d8ee5f025f02594c6152ec_1.jpeg)

## <font color='red'>Required Packages</font>
We will need the Python packages:

- `requests`: for accessing servers and getting the contents of web pages.
- `json`: for manipulating JSON documents.
- `BeautifulSoup`: for pulling information out of HTML and XML files. 
- `Pandas`: for data analysis and manipulation.
- `Matplotlib`: for creating visualization.
- `Seaborn`: for drawing attractive and informative statistical graphics.
- `Plotly`: for making interactive graphs.

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
import os
import pprint
import re
import io

In [ ]:
import requests as reqs

In [ ]:
import json

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
import seaborn as sns

In [ ]:
import plotly.express as px

In [ ]:
from bs4 import BeautifulSoup as bso

In [ ]:
from IPython.display import HTML

In [ ]:
#print(f"Requests version:  {reqs.__version__}")
#print(f"JSON version:      {json.__version__}")

# <font color='red'>Accessing web servers with `requests` </font>

* Requests is a built-in Python module.
* Requests is a simple and elegant Python HTTP (Hypertext Transfer Protocol) library. 
* It provides methods for accessing Web resources via HTTP. 
* The HTTP request returns a Response Object with all the response data (content, encoding, status, etc.).

**Reading a Web Page**
- We use the function `get()` to grab the content of a web page into an object.
- We extract from the object the HTML content of the page.

### Content-Type

- It is part of the HTTP header.
   - A string used to indicate the media type of the resource you want to access.  
   - It tells the browser the type of content it has to load on the machine. 
- Here are some values of `content-type`:

```html
   text/html
   text/csv
   application/json
   application/javascript
   audio/ogg
   image/png
```

- While scraping a web page, it is used to determine which tool is needed to parse the content of a web page.

#### Function to accesss a web page
We want to pass the `payload` and `timeout` as arguments of the function.

In [ ]:
def access_website(url: str, payload: dict=None, timeout: int=10):
    """
    Attempt to access a server. If the attempt is successful,
    return the response object, otherwise return an error message.

    Parameters
    ----------
    url : str
       HTTP address of the web page we want to access
    payload : dict
       Parameters needed to construct the target url.
    timeout: int
       Maximum time (in seconds) to access a website and grab its content.

    Returns
    -------
    resp : object
       Object which has infomation on the web page of interest.
    """
    try:
        if payload:
            resp = reqs.get(url, params=payload, timeout=timeout)
        else:
            resp = reqs.get(url, timeout=timeout)
        resp.raise_for_status()
    except reqs.exceptions.HTTPError as errh:
        print(f"Http Error: {errh}")
    except reqs.exceptions.ConnectionError as errc:
        print(f"Error Connecting: {errc}")
    except reqs.exceptions.Timeout as errt:
        print(f"Timeout Error: {errt}")
    except reqs.exceptions.RequestException as err:
        print(f"General Error: {err}")
    else:
        print(f"Successfully accessed the site: \n\t {resp.url}")
        print(f"Content type: \n\t {resp.headers['content-type']}")
    return resp

---

## <font color="red">Accessing the DONKI webservice API</font>

The Space Weather Database Of Notifications, Knowledge, Information (DONKI), developed at the Community Coordinated Modeling Center (CCMC), is a comprehensive on-line tool for space weather forecasters, scientists, and the general space science community.
It serves as an archive for space weather activities including solar flares, coronal mass ejections (CMEs), solar energetic particles, and geomagnetic storms.

DONKI provides:
- Chronicles the daily interpretations of space weather observations, analysis, models output, and notifications provided by the Moon to Mars Space Weather Analysis Office as a courtesy to the community.
- Comprehensive knowledge-base search functionality to support anomaly resolution and space science research.
- Intelligent linkages, relationships, cause-and-effects between space weather activities.
- Comprehensive webservice API access to information stored in DONKI

__Our goal__:
- We want to be able to obtain from the webpage [https://kauai.ccmc.gsfc.nasa.gov/DONKI/WS/get/CME](https://kauai.ccmc.gsfc.nasa.gov/DONKI/WS/get/CME) the Coronal Mass Ejection (CME) data
- We access the webpage (using a set of parameters) and retrieve the content of the page as a JSON object.


### <font color="blue">Access CME Web Database Server</font>

The based URL is: `https://kauai.ccmc.gsfc.nasa.gov/DONKI/WS/get/CME`

To access the data, we need to provide two parameters:

- `startDate` (YYYY-MM-DD): default to 30 days prior to current UTC date
- `endDate` (YYYY-MM-DD): default to current UTC date

In return, we get a JSON object containing the following fields:

- `activityID`
- `startTime`
- `sourceLocation`
- `activeRegionNum`
- `instruments`
- `cmeAnalyses`
- `linkedEvents[(list of) activityID]`
- `note`
- `catalog`




In [ ]:
cme_url = "https://kauai.ccmc.gsfc.nasa.gov/DONKI/WS/get/CME"

In [ ]:
startDate="2017-01-01"
endDate="2017-03-31"

payload = {
    'startDate': startDate,
    'endDate': endDate,
}

cme_response = access_website(cme_url, payload, timeout=150)

### <font color="blue">Process the data with JSON</font>

- We have a JSON object that we need to convert into a Python object.
- We need to use the `loads()` function (deserialization).

In [ ]:
cme_activities = json.loads(cme_response.text)

Whenever we have a Python object, it is good to determine its type before proceeding.

In [ ]:
type(cme_activities)

What is the length of the list?

In [ ]:
len(cme_activities)

Let us look at the first entry of the list:

In [ ]:
pprint.pprint(cme_activities[0])

It seems that there is was no activity on `2017-01-01T00:39Z`: 

```
  'cmeAnalyses': None
```

In [ ]:
one_cme_activity = cme_activities[1]

In [ ]:
pprint.pprint(one_cme_activity)

### <font color="blue">Basic data extraction from one CME activity</font>

We need to first get the keys of the dictionary:

In [ ]:
one_cme_activity.keys()

We can focus on:

- `'cmeAnalyses'`
- `'startTime'`
- `'instruments'`

__Get instruments__

In [ ]:
one_cme_activity['instruments']

In [ ]:
for i, item in enumerate(one_cme_activity['instruments'], start=1):
    print(f"Instrument {i}- {item['displayName']}")

__Get start date/time__

In [ ]:
one_cme_activity['startTime']

__Get CME activity data__

In [ ]:
pprint.pprint(one_cme_activity['cmeAnalyses'])

We want to be able to extract the following parameters:
- `speed`
- `longitude`
- `latitude`
- `halfAngle`

In [ ]:
type(one_cme_activity['cmeAnalyses'])

In [ ]:
len(one_cme_activity['cmeAnalyses'])

In [ ]:
pprint.pprint(one_cme_activity['cmeAnalyses'][0])

In [ ]:
latitude = one_cme_activity['cmeAnalyses'][0]['latitude']
longitude = one_cme_activity['cmeAnalyses'][0]['longitude']
halfAngle = one_cme_activity['cmeAnalyses'][0]['halfAngle']
speed = one_cme_activity['cmeAnalyses'][0]['speed']

In [ ]:
print(f"  latitude: {latitude}")
print(f" longitude: {longitude}")
print(f" halfAngle: {halfAngle}")
print(f"     speed: {speed}")

In [ ]:
one_cme_activity['instruments']

In [ ]:
for item in one_cme_activity['instruments']:
    print(item['displayName'])

### <font color="blue">Data Analysis</font>

- We want to inspect the obtained Python list and identify the dates/times when there were CME activities.
- For each date/time when there was an activity, we will extract from the list:
    - The `date/time`, the `speed`, the latitude, the longitude, and the hall angle
    - The the list of instruments 

In [ ]:
def get_CME_parameters(cme_activity: dict):
    """
    From a dictionary containing CME activity data on a specific date/time,
    extract the following parameters:

    - starting date/time
    - latitude
    - longitude
    - halfAngle
    - speed
    - instruments
    """

    latitude = cme_activity['cmeAnalyses'][0]['latitude']
    longitude = cme_activity['cmeAnalyses'][0]['longitude']
    half_angle = cme_activity['cmeAnalyses'][0]['halfAngle']
    speed = cme_activity['cmeAnalyses'][0]['speed']
    start_time = cme_activity['startTime']
    list_instruments = list()
    for item in cme_activity['instruments']:
        list_instruments.append(item['displayName'])
    return start_time, speed, latitude, longitude, half_angle, tuple(sorted(list_instruments))

In [ ]:
get_CME_parameters(one_cme_activity)

In [ ]:
def create_cme_df(activities: list):
    """
    Using a list of CME activities, create a Pandas DataFrame
    with columns:
    
    - star_time
    - latitude
    - longitude
    - half_angle
    - speed
    - instruments
    """
    columns = ["start_time", "speed", "latitude", "longitude", "half_angle", "instruments"]
    df = pd.DataFrame(columns=columns)

    # Loop over the CME event
    for activity in activities:
        # Only process when a CME activity was recorded.
        if activity['cmeAnalyses']:   # This mean that an activity was recorded
            df.loc[len(df)] = get_CME_parameters(activity)
    return df     

In [ ]:
cme_df = create_cme_df(cme_activities)

In [ ]:
cme_df

In [ ]:
cme_df.info()

In [ ]:
cme_df.describe()

We need to convert the `start_time` into a datetime object:

In [ ]:
cme_df['start_time'] = pd.to_datetime(cme_df['start_time'])

In [ ]:
cme_df

Set the `start_time` column as index of the Pandas DataFrame:

In [ ]:
cme_df.set_index('start_time', inplace=True)
cme_df

#### Write a function that accesses the DONKI API and collects CME activities

In [ ]:
def collect_cme_activities(start_date: str, end_date: str):
    """
    Use web scraping to gather from DONKI all the CME activities
    between the beginning date (start_date) and the end date (end_date).

    Parameters
    ----------
    start_date : str
       Beginning date in the format YYYY-MM-DD
    end_date : str
       End date in the format YYYY-MM-DD

    Returns
    -------
    df : Pandas DataFrame
       Pandas DataFrame containg parameters of CME activities.
    """
    cme_url = "https://kauai.ccmc.gsfc.nasa.gov/DONKI/WS/get/CME"
    payload = {
        'startDate': start_date,
        'endDate': end_date
    }

    cme_response = access_website(cme_url, payload, timeout=150)
    cme_activities = json.loads(cme_response.text)
    cme_df = create_cme_df(cme_activities)

    cme_df['start_time'] = pd.to_datetime(cme_df['start_time'])
    cme_df.set_index('start_time', inplace=True)
    return cme_df

In [ ]:
cme_df = collect_cme_activities("2017-01-01", "2019-12-31")

In [ ]:
cme_df

__Get basic information on columns using the `info()` method__

In [ ]:
cme_df.info()

__Observations__: 

- The values of each column have the same data type.
- There are 330 data points (rows).
- The columns `latitude` and `longitude` have 329 and 325 values respectively.
   - The two columns have missing values.
   - If the two columns are important for your analysis, you need to decide what to do with missing values.

__To get the number of missing values in each column:__

In [ ]:
cme_df.isnull().sum()

__Basic descriptive statistics__

In [ ]:
cme_df.describe()

### <font color="green">Focus on the instruments</font>

__Get the the unique instruments__

In [ ]:
set_instruments = set()

for item in cme_df['instruments'].values:
    for inst in item:
        set_instruments.add(inst)

print(f"Set of instruments: \n\t {sorted(list(set_instruments))}")

__Combined sets of instruments__

In [ ]:
df_instr = cme_df.groupby('instruments')

In [ ]:
df_instr

In [ ]:
df_instr.groups.keys()

In [ ]:
df_instr.count()["speed"]

In [ ]:
plt.pie(df_instr.count()["speed"], 
        labels=list(df_instr.groups.keys()), 
         autopct='%.0f%%') 
plt.title("Pie Chart of Unique Combined Sets of Instruments");

In [ ]:
fig = px.pie(df_instr.count(), 
             values="speed",
             names=list(df_instr.groups.keys()),
             title="Pie Chart of Unique Combined Sets of Instruments",
            width=700)
fig.show()

__Count the number times each instrument is used__

We will create a dictionary with key as the instrument name and value the number of times used.

In [ ]:
dict_instruments = dict()

for item in cme_df['instruments'].values:
    for inst in item:
        if inst in dict_instruments:
            dict_instruments[inst] += 1
        else:
            dict_instruments[inst] = 1

In [ ]:
dict_instruments

In [ ]:
plt.bar(dict_instruments.keys(), dict_instruments.values());

In [ ]:
dict_instruments.values()

In [ ]:
plt.pie(list(dict_instruments.values()), 
        labels=list(dict_instruments.keys()), 
         autopct='%.0f%%') 
plt.title("How many times instruments are used");

### <font color="green"> Plot time series values of the speed</font>

Using Matplotlib:

In [ ]:
fig, ax = plt.subplots(figsize=(12, 7))
ax.scatter(cme_df.index, cme_df["speed"], s=7)
ax.set_xlabel("CME Start Date")
ax.set_ylabel("CME Speed (km/s)")
ax.set_title(f"DONKI CME Database \n Date vs CME Speed")
ax.grid()

Using Seaborn:

In [ ]:
fig = plt.figure(figsize=(12, 7))
ax = sns.scatterplot(x=cme_df.index, y=cme_df['speed'])
ax.set_xlabel("CME Start Date")
ax.set_ylabel("CME Speed (km/s)")
ax.set_title(f"DONKI CME Database \n Date vs CME Speed")
ax.grid()

Using Plotly (interactive plot):

In [ ]:
fig = px.scatter(x=cme_df.index, y=cme_df['speed'],
                width=800, height=600)
fig.show()

__Create histograms__

In [ ]:
cme_df["speed"].hist(bins=5)

In [ ]:
fig = px.histogram(cme_df, x="speed", nbins=5)
fig.show()

__Create a new column to categorize speed into ranges__

| Type | Speed range |
| --- | --- |
| `S-type` | $<500$ km/s |
| `C-type` | $500-999$ km/s |
| `0-type` | $1000-1999$ km/s |
| `R-type` | $2000-2999$ km/s |
| `ER-type` | $>3000$ km/s |

In [ ]:
speed_bins = [0, 500, 1000, 2000, 3000, float('inf')]
speed_labels = [
    'S-type (<500 km/s)', 
    'C-type (500-999 km/s)', 
    'O-type (1000-1999 km/s)', 
    'R-type (2000-2999 km/s)', 
    'ER-type (>3000 km/s)'
]
assert(len(speed_bins)-1 == len(speed_labels))

In [ ]:
cme_df['speed_range'] = pd.cut(
    cme_df['speed'], 
    bins=speed_bins,
    labels=speed_labels
)
cme_df

In [ ]:
fig = px.histogram(cme_df, x="speed_range", color="speed_range")
fig.show()

In [ ]:
fig = px.bar(cme_df, x='speed_range', y='speed', color="speed_range", text='speed', height=600)
fig.show()

In [ ]:
fig = px.bar(cme_df, x=cme_df.index.year, y='speed', color="speed_range", height=600)
fig.show()

#### Plot the time series values of speed by categories

In [ ]:
fig = plt.figure(figsize=(12, 7))
ax = sns.scatterplot(data=cme_df, x=cme_df.index, y='speed', hue='speed_range')
ax.set_xlabel("CME Start Date")
ax.set_ylabel("CME Speed (km/s)")
ax.set_title(f"DONKI CME Database \n Date vs CME Speed")
ax.grid()

#### Longitude vs Speed plot (polar plot)

In [ ]:
g = sns.FacetGrid(cme_df, subplot_kws=dict(projection='polar'), 
                  height=5, sharex=False, sharey=False, despine=False)
g.map_dataframe(sns.scatterplot, 
                x='longitude', 
                y='speed', 
                hue='speed_range');

We can generate the same polar plot with `Plotly` (note the interactivity):

In [ ]:
fig = px.scatter_polar(cme_df, 
                       r='speed', 
                       theta='longitude', 
                       color='speed_range', 
                       width=700, height=700)

#fig.update_layout(showlegend=False)
fig.show()

### <font color="green">Gather statistics</font>

__CME for a specific month__

In [ ]:
cme_df[cme_df.index.month==2]

In [ ]:
month_count_df = pd.DataFrame(columns=["Month", "Count"])
for i in range(1,13):
    month_count_df.loc[len(month_count_df)] = [i, len(cme_df[cme_df.index.month==i])]
month_count_df

In [ ]:
month_count_df["Month"] = ["Jan", "Feb", "Mar", "Apr", "May", "Jun", 
                           "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]

In [ ]:
# Create the bar plot
sns.barplot(data=month_count_df, x='Month', y='Count', hue="Count", legend=False)

# Add title and labels
#plt.title('Bar Chart Example')

In [ ]:
fig = px.bar(month_count_df, x="Month", y='Count', height=500)
fig.show()

__Data on a specific day__

In [ ]:
cme_df.loc['2019-05-13']

__Data over a specific range of date__

In [ ]:
cme_df.loc['2018-01-01 09:00:00':'2018-04-14 11:00:00']

__Data for a specific year/month__

In [ ]:
cme_df.loc['2018-01']

__Data with a specific speed__

In [ ]:
cme_df[cme_df.speed==350.0]

__Speed monthly and yearly means__

In [ ]:
speed_monthly_means = cme_df.speed.resample('M').mean()

In [ ]:
speed_monthly_means.plot()

In [ ]:
cme_df.speed.resample('Y').mean()

__Group by speed range__

In [ ]:
df_range = cme_df.groupby('speed_range')

In [ ]:
list_groups = list(df_range.groups.keys())
list_groups

In [ ]:
df_range.count()["speed"]

In [ ]:
df_S = df_range.groups['S-type (<500 km/s)']

In [ ]:
df_S

In [ ]:
len(df_S[df_S.slice_indexer("2017-01-01", "2017-12-31")])

In [ ]:
len(df_S[df_S.slice_indexer("2018-01-01", "2018-12-31")])

In [ ]:
len(df_S[df_S.slice_indexer("2019-01-01", "2019-12-31")])

In [ ]:
fig = px.pie(df_range.count(), values="speed", names=list_groups)
fig.show()

---
---

# <font color="red">Search and display CME related GIF images</font>

Here we focus on a one-day CME activity and perform web scraping to extract the gif images generated on the activity.

In [ ]:
startDate="2017-01-03"
endDate="2017-01-03"

payload = {
    'startDate': startDate,
    'endDate': endDate,
}

page_content = access_website(cme_url, payload, timeout=15)

#### Task 1: Get link associated with the `enlilList` key

In [ ]:
page = json.loads(page_content.text)[0]

In [ ]:
page['cmeAnalyses']

In [ ]:
pprint.pprint(page['cmeAnalyses'][0])

__Extract the link of interest__

In [ ]:
url_link = page['cmeAnalyses'][0]['enlilList'][0]['link']
url_link

#### Task 2: Get the gif images contain in the webpage

__Access the webpage and get its content__

In [ ]:
source = access_website(url_link)

__Use BeautifulSoup to obtain the HTML document__

In [ ]:
mysoup = bso(source.text, 'html.parser')

In [ ]:
print(mysoup.prettify())

__Parse the content of the HTML document to extract all the web links that have gif images__

In [ ]:
list_images = [a['href'] for a in mysoup.select('a[href]') if "iswa" in a['href']]

In [ ]:
len(list_images)

In [ ]:
list_images

#### Task 3: Display some of the remote images

In [ ]:
HTML(f'<img src={list_images[0]}>')

In [ ]:
HTML(f'<img src={list_images[1]}>')

In [ ]:
HTML(f'<img src={list_images[2]}>')

In [ ]:
HTML(f'<img src={list_images[3]}>')

In [ ]:
HTML(f'<img src={list_images[7]}>')

---

# <font color="red">Extract the monthly sunspot numbers</font>

The Bureau of Meteorology Australian Space Weather Forecasting Centre provides monthly sunspot numbers at:

[https://www.sws.bom.gov.au/Solar/1/6](https://www.sws.bom.gov.au/Solar/1/6)

The values in the smoothed sunspot number table are based on the observed    
(definitive and provisional) monthly mean sunspot numbers from WDC-SILSO,    
Royal Observatory of Belgium, Brussels ([sidc.oma.be/silso/home](sidc.oma.be/silso/home)).   

We want to access the above webspage and extract data from the `SMOOTHED SUNSPOT NUMBER` table.

__Access the website and extract its content__

In [ ]:
solar_url = "https://www.sws.bom.gov.au/Solar/1/6"
solar_resp = access_website(solar_url)

In [ ]:
solar_soup = bso(solar_resp.text, 'html.parser')

In [ ]:
solar_tables = solar_soup.select('p.file-text')

In [ ]:
solar_tables[0].text

__Replace on the tables the character `e` by empty space__

In [ ]:
def remove_char_e(ref_str: str) -> str:
    list_str = ref_str.split("\n")
    new_list = list()
    for line in list_str:
        try:
            a = int(line[:4])
        except:
            a = line[:4]
        if isinstance(a, int) and 'e' in line:
            new_list.append(line.replace('e', ' ').strip())
        else:
            new_list.append(line.strip())
    return  "\n ".join(new_list)   

In [ ]:
remove_char_e(solar_tables[0].text)

__Extract the first table and create a Pandas DataFrame__

In [ ]:
file_object = io.StringIO(remove_char_e(solar_tables[0].text))
solar_df = pd.read_csv(
    file_object, 
    sep=r"\s+", 
    comment="-",
    skiprows=7,           # skip the first 3 rows
    skipfooter=58,         # skip the last 7 rows
    #index_col=0,          # make the first column as index (labels of the rows)
    #na_values="*****",    # deal with missing values
    engine='python'
)

solar_df

In [ ]:
solar_df[list(solar_df.columns[1:])].describe().T

__Create a new DataFrame with only `Sunspots` as column__

In [ ]:
sunspot_df = solar_df.melt(
    id_vars='Year', 
    var_name='month', 
    value_name='Sunspots'
)
sunspot_df

In [ ]:
# create a datatime column and set it as index
sunspot_df['date'] = pd.to_datetime(sunspot_df['Year'].astype(str) + '-' + sunspot_df['month'])
sunspot_df.set_index('date', inplace=True)

In [ ]:
sunspot_df

In [ ]:
sunspot_df.drop(["Year", "month"], axis=1, inplace=True)
sunspot_df

In [ ]:
sunspot_df.info()

In [ ]:
sunspot_df = sunspot_df.sort_index()
sunspot_df

In [ ]:
sunspot_df.describe()

__Visualize the data__

In [ ]:
solar_df.set_index('Year').plot(figsize=(10, 13), subplots=True);

In [ ]:
fig, ax = plt.subplots(figsize=(7,4))
fig.suptitle('Line Plot of the Time Series Data')
sunspot_df.plot(ax=ax);

In [ ]:
fig, ax = plt.subplots(figsize=(7,4))
fig.suptitle('Histogram of the Time Series Data')
sunspot_df.hist(ax=ax);

# <font color="red"> Breakout</fon>

This exercise is inspired by:


[Project 1: Data scraping and cleaning](https://www.cs.umd.edu/class/spring2017/cmsc320/hw/Project1.html)

A possible solution can be seen at:

[NASA Data Scraping, Cleaning, & Data Analysis](https://www.kaggle.com/code/fareselsalamouny/nasa-data-scraping-cleaning-data-analysis)


## <font color="blue"> Part 1:</font> Scrape data for the top 50 solar flares

Access the website:

[https://www.spaceweatherlive.com/en/solar-activity/top-50-solar-flares.html](https://www.spaceweatherlive.com/en/solar-activity/top-50-solar-flares.html)

to create a Pandas DataFrame.


In [ ]:
import http.client as http_client
http_client.HTTPConnection.debuglevel = 0 #1

In [ ]:
#spaceweather_url = "https://www.spaceweatherlive.com/en/solar-activity/top-50-solar-flares"

In [ ]:
spaceweather_url = "https://www.spaceweatherupdate.com/en/solar-activity/top-50-solar-flares/solar-cycle/25.html"

In [ ]:
spaceweather_resp = access_website(spaceweather_url)

In [ ]:
spaceweather_soup = bso(spaceweather_resp.content, 'html.parser')

In [ ]:
#spaceweather_soup

In [ ]:
print(spaceweather_soup.prettify())

__Extract the HTML table from the web document__

In [ ]:
html_table = spaceweather_soup.find('table')

In [ ]:
type(html_table)

__Create a DataFrame__

In [ ]:
spaceweather_columns = [
    'rank', 
    'x_class', 
    'date', 
    'region', 
    'start_time', 
    'max_time', 
    'end_time', 
    'movie'
]

In [ ]:
spaceweather_df = pd.DataFrame(columns=spaceweather_columns, index = range(0,50))

# Loop over rows of HTML table to extract the text and populate the DataFrame
row_marker = 0
for row in html_table.find_all('tr'):
    column_marker = 0
    columns = row.find_all('td')
    row_marker += 1
    for column in columns:
        if (row_marker > 1):
            spaceweather_df.iat[row_marker-2,column_marker] = column.get_text()
            column_marker += 1

In [ ]:
spaceweather_df

__Drop the last column (`movie`)__

In [ ]:
spaceweather_df = spaceweather_df.drop(spaceweather_columns[-1], axis=1)

In [ ]:
spaceweather_df.info()

__Combine the date column with the time columns to convert to datetime format__

In [ ]:
spaceweather_df['start_time'] = pd.to_datetime(spaceweather_df['date'] + ' ' + spaceweather_df['start_time'])

In [ ]:
spaceweather_df['max_time'] = pd.to_datetime(spaceweather_df['date'] + ' ' + spaceweather_df['max_time'])

In [ ]:
spaceweather_df['end_time'] = pd.to_datetime(spaceweather_df['date'] + ' ' + spaceweather_df['end_time'])

__Drop the `date` column__

In [ ]:
spaceweather_df.drop('date', axis=1)

__Change the order of the columns__

In [ ]:
spaceweather_df = spaceweather_df[['rank', 'x_class', 'start_time', 'max_time', 'end_time', 'region']]

__Replace all instances of `'-'` with `'<NA>'`__

In [ ]:
spaceweather_df = spaceweather_df.replace('-', '<NA>')

In [ ]:
spaceweather_df

## <font color="blue"> Part 2:</font> Scrape the NASA data

Access the webpage:

[https://cdaw.gsfc.nasa.gov/CME_list/radio/waves_type2.html](https://cdaw.gsfc.nasa.gov/CME_list/radio/waves_type2.html)

to scrap the STEREO and Wind WAVES type II bursts and the associated CMEs. 

In [ ]:
nasa_url = "https://cdaw.gsfc.nasa.gov/CME_list/radio/waves_type2.html"

In [ ]:
nasa_resp = access_website(nasa_url)

In [ ]:
nasa_bs = bso(nasa_resp.content, 'html.parser')

In [ ]:
#print(nasa_bs.prettify())

__Extract the HTML table__

In [ ]:
nasa_table = nasa_bs.find('pre').text

In [ ]:
#print(nasa_table)

__Convert the table to a list__

- Do not take the first 11 rows of the table.
- Do not take any row that has `===`.

In [ ]:
nasa_table_list = [x.split() for x in nasa_table.split('\n')[11:] if "===" not in x and x]
#nasa_table_list

__Create the DataFrame__

- Only consider the first 14 columns.

In [ ]:
nasa_df = pd.DataFrame(nasa_table_list).loc[:,0:13]

In [ ]:
nasa_df

In [ ]:
nasa_df.info()

__Provide the labels for the columns__

In [ ]:
columns = [
    "start_date", #Column 1: Starting date of the type II burst (yyyy/mm/dd format)
    "start_time", #Column 2:  Starting time (UT) of the type II burst (hh:mm format)
    "end_date", #Column 3:  Ending date of the type II burst (mm/dd format; year in Column 1 applies)
    "end_time", #Column 4:  Ending time of the Type II burst (hh:mm  format)
    "start_frequency", #Column 5:  Starting frequency of type II burst (kHz) [1]
    "end_frequency", #Column 6:  Ending frequency of type II burst (kHz) [1]
    "flare_location", #Column 7:  Solar source location (Loc) of the associated eruption in heliographic coordinates [2]
    "flare_region",  #Column 8:  NOAA active region number (NOAA) [3]
    "flare_classification",  #Column 9:  Soft X-ray flare importance (Imp)  [4]
    "cme_date",   #Column 10: Date of the associated CME (mm/dd format, Year in Column 1 applies) [5]
    "cme_time",   #Column 11: Time of the associated CME (hh:mm format)
    "cme_angle",  #Column 12: Central position angle (CPA, degrees) for non-halo CMEs [6]
    "cme_width",  #Column 13: CME width in the sky plane (degrees) [7]
    "cme_speed"   #Column 14: CME speed in the sky plane (km/s)
]

nasa_df.columns = columns
nasa_df

__Condition [1]__

In [ ]:
nasa_df["start_frequency"] = nasa_df["start_frequency"].apply(lambda x: None if '?' in x else x)

In [ ]:
nasa_df["end_frequency"] = nasa_df["end_frequency"].apply(lambda x: None if '?' in x else x)

__Condition [3]__

In [ ]:
nasa_df["flare_region"] = nasa_df["flare_region"].apply(lambda x: None if '--' in x else x)

In [ ]:
nasa_df.head(6)

__Condition [4]__

In [ ]:
#nasa_df["flare_classification"] = nasa_df["flare_classification"].apply(lambda x: None if '--' in x else x)

Only get the rows that have `X` in classification flare

In [ ]:
flares_df = nasa_df.loc[nasa_df['flare_classification'].str.contains('X')]

In [ ]:
flares_df

Get rid of X for sorting purposes

In [ ]:
flares_df['flare_classification'] = flares_df['flare_classification'].str.lstrip('X')

Convert to float

In [ ]:
flares_df['flare_classification'] = flares_df.flare_classification.astype(float)

Sort rows by flare class

In [ ]:
flares_df = flares_df.sort_values('flare_classification', ascending = False)

In [ ]:
flares_df

__Only consider the top 50__

In [ ]:
flares_df = flares_df.head(50)

Reconvert back to string

In [ ]:
flares_df['flare_classification'] = flares_df.flare_classification.astype(str)

Add the `X` before any classification value

In [ ]:
flares_df['flare_classification'] = "X" + flares_df['flare_classification']

In [ ]:
flares_df